In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
# Define directories
train_dir = 'C:/Users/DELL/Downloads/archive (6)/train'
valid_dir = 'C:/Users/DELL/Downloads/archive (6)/val'
test_dir = 'C:/Users/DELL/Downloads/archive (6)/test'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 4183 images belonging to 3 classes.
Found 208 images belonging to 3 classes.
Found 837 images belonging to 3 classes.


In [11]:
def create_model(base_model):
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))  # 3 classes: PNEUMONIA, NORMAL, COVID
    return model

# Load base models without the top layer
base_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_vgg.layers:
    layer.trainable = False
for layer in base_resnet.layers:
    layer.trainable = False
for layer in base_mobilenet.layers:
    layer.trainable = False

# Create models
vgg_model = create_model(base_vgg)
resnet_model = create_model(base_resnet)
mobilenet_model = create_model(base_mobilenet)

# Compile models
for model in [vgg_model, resnet_model, mobilenet_model]:
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Train models
vgg_model.fit(train_generator, validation_data=valid_generator, epochs=20, callbacks=[early_stopping])
resnet_model.fit(train_generator, validation_data=valid_generator, epochs=20, callbacks=[early_stopping])
mobilenet_model.fit(train_generator, validation_data=valid_generator, epochs=20, callbacks=[early_stopping])


In [13]:
# Save the models after training
vgg_model.save('vgg_model.h5')
resnet_model.save('resnet_model.h5')
mobilenet_model.save('mobilenet_model.h5')

In [14]:
from tensorflow.keras.models import load_model
# Load the models
vgg_model = load_model('vgg_model.h5')
resnet_model = load_model('resnet_model.h5')
mobilenet_model = load_model('mobilenet_model.h5')

In [15]:
def evaluate_model(model, test_generator):
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f'Test Accuracy: {test_accuracy:.4f}')
    return test_accuracy

vgg_accuracy = evaluate_model(vgg_model, test_generator)
resnet_accuracy = evaluate_model(resnet_model, test_generator)
mobilenet_accuracy = evaluate_model(mobilenet_model, test_generator)

# Compare accuracies
accuracies = {
    'VGG16': vgg_accuracy,
    'ResNet50': resnet_accuracy,
    'MobileNetV2': mobilenet_accuracy
}

print("Model Accuracies:")
for model, accuracy in accuracies.items():
    print(f"{model}: {accuracy:.4f}")


27/27 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - accuracy: 0.9846 - loss: 0.0893
Test Accuracy: 0.9665
27/27 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.7714 - loss: 0.7639
Test Accuracy: 0.6989
27/27 ━━━━━━━━━━━━━━━━━━━━ 27s 800ms/step - accuracy: 0.9714 - loss: 0.0901
Test Accuracy: 0.9582
Model Accuracies:
VGG16: 0.9665
ResNet50: 0.6989
MobileNetV2: 0.9582


In [16]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
def preprocess_image(image_path):
    img = Image.open(image_path).resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict_image(image_path):
    img_array = preprocess_image(image_path)
    vgg_pred = vgg_model.predict(img_array)
    resnet_pred = resnet_model.predict(img_array)
    mobilenet_pred = mobilenet_model.predict(img_array)
    avg_pred = (vgg_pred + resnet_pred + mobilenet_pred) / 3
    predicted_class = np.argmax(avg_pred, axis=1)
    class_labels = list(train_generator.class_indices.keys())
    predicted_label = class_labels[predicted_class[0]]
    return predicted_label

def upload_image():
    file_path = filedialog.askopenfilename(title="Select an Image", 
                                            filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        img = Image.open(file_path).resize((200, 200))
        img_tk = ImageTk.PhotoImage(img)
        img_label.config(image=img_tk)
        img_label.image = img_tk
        prediction = predict_image(file_path)
        messagebox.showinfo("Prediction", f"The image is predicted to be: {prediction}")

# Create the main Tkinter window
root = tk.Tk()
root.title("X-ray Image Prediction")
root.configure(bg="black")
# Add a button to upload an image
upload_button = tk.Button(root, text="Upload X-ray Image", command=upload_image, bg="black", fg="white")
upload_button.pack(pady=20)

# Label to display the image
img_label = tk.Label(root)
img_label.pack(pady=10)

# Run the Tkinter event loop
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [ ]:
ki